# Code

In [3]:
# Imports and setup model
import os
import openai
import json
import configobj
import tiktoken
from time import sleep

In [4]:
# Setup env vars and constants
config = configobj.ConfigObj('.env')
openai.api_key = config["OPENAI_API_KEY"]
pinecone_key = config["PINECONE_API_KEY"]
pinecone_env = config["PINECONE_ENVIRONMENT"]

embedding_model = "text-embedding-ada-002" 
api_model = "gpt-3.5-turbo-16k"
dimensions = 1536
docs_dir = 'docs/'
typed_keys = ['arcs', 'geos', 'nodes', 'stats']

In [5]:
#Setup Tokenizer
tokenizer_name = tiktoken.encoding_for_model(api_model)
tokenizer = tiktoken.get_encoding(tokenizer_name.name)

# create the length function
def tiktoken_len(text):
    tokens = tokenizer.encode(
        text,
        disallowed_special=()
    )
    return len(tokens)


In [6]:
# load docs
texts = []
docs_by_name = {}

for doc in os.listdir(docs_dir):
    with open(docs_dir + doc, 'r') as f:
        text = f.read()
        texts.append(text)
        docs_by_name[doc[0:-3]] = text

print(texts[1])

### `maps`
This key group allows designers to specify information about the starting state of the map, what information is contained and how it is grouped in the legend, and what viewports can be easily jumped to by the user.

## Common keys
- [`allowModification`](../common_keys/common_keys.md#allowModification)
- [`data`](../common_keys/common_keys.md#data)
- [`icon`](../common_keys/common_keys.md#icon)
- [`name`](../common_keys/common_keys.md#name)
- [`order`](../common_keys/common_keys.md#order)
- [`sendToApi`](../common_keys/common_keys.md#sendToApi)
- [`sendToClient`](../common_keys/common_keys.md#sendToClient)

## Special and custom keys
Key | Default | Description
--- | ------- | -----------
<a name="defaultViewport">`customMapKey.defaultViewport`</a> | | A dictionary object containing geo properties that set the map's default field of view. Also used by the "home" viewport button in the app.
`customMapKey.defaultViewport.bearing` | `0` | The initial bearing (rotation) of the m

In [7]:
state = ""
with open('state.json') as f:
    state = f.read()

In [8]:
topLevel = ""
with open(f'{docs_dir}topLevelInfo.md') as f:
    topLevel = f.read()

print(tiktoken_len(topLevel))

916


In [14]:
query = "Center the map on China"

# Run Query through gpt4

Note: Currently using gpt-4 as gpt-4-32k hasn't been publicly released yet and gpt-3.5-turbo-16k isn't great

In [15]:
completion = openai.ChatCompletion.create(
  model=api_model,
  temperature=0.1,
  messages=[
    {"role": "system", "content": "Using the given documentation delimited by triple quotes, respond to user requests with nothing but the name of the top-level key that must be edited in order to fufill the request surrounded by single quotes"},
    {"role": "user", "content": '"""' + topLevel + '""" \n\n' + 'request: ' + query},
  ]
)
current_key = completion.choices[0].message.content.replace("'", '')

print(current_key)

maps


In [16]:
encoded_data = json.dumps(json.loads(state)[current_key])
max_size = False
chunks = [None]
if tiktoken_len(encoded_data) > 7000:
    max_size = True
    key_dict = json.loads(state)[current_key]
    chunks[-1] = {i:key_dict[i] for i in key_dict if i!='data'}
    chunks[-1]['data'] = {}
else:
    chunks = [encoded_data]
print(tiktoken_len(json.dumps(chunks)))
if max_size:
    for key, value in key_dict['data'].items():
        if tiktoken_len(json.dumps(chunks[-1]) + json.dumps({key: value})) > 7000:
            chunks.append({i:key_dict[i] for i in key_dict if i!='data'})
            chunks[-1]['data'] = {key: value}
        else:
            chunks[-1]['data'][key] = value      
    for idx, chunk in enumerate(chunks):
        chunks[idx] = json.dumps(chunk)

for data in chunks:
    print(tiktoken_len(data))

print(len(chunks))

496
470
1


In [17]:
chunks[0]

'{"data": {"map1": {"defaultViewport": {"name": "Central Europe", "icon": "FaGlobeEurope", "longitude": 10, "latitude": 45, "zoom": 5, "pitch": 0, "bearing": 0, "height": 1287, "altitude": 1.5, "maxZoom": 12, "minZoom": 2, "order": 3}, "optionalViewports": {"ovNA": {"name": "North America", "icon": "FaGlobeAmericas", "longitude": -75, "latitude": 40, "zoom": 4, "pitch": 0, "bearing": 0, "height": 1287, "altitude": 1.5, "maxZoom": 12, "minZoom": 2, "order": 1}, "ovEUCentral": {"name": "Central Europe", "icon": "FaGlobeEurope", "longitude": 10, "latitude": 45, "zoom": 5, "pitch": 0, "bearing": 0, "height": 1287, "altitude": 1.5, "maxZoom": 12, "minZoom": 2, "order": 3}}, "legendGroups": {"Network Flows": {"name": "Network Flows", "arcs": {"Road": {"value": true, "colorBy": "modeColor", "sizeBy": "quantity"}, "Rail": {"value": true, "colorBy": "modeColor", "sizeBy": "quantity"}}}, "Network Facilities": {"name": "Network Facilities", "nodes": {"nodeTypeRT": {"value": true, "colorBy": "acti

In [18]:
for data in chunks:
  completion = openai.ChatCompletion.create(
    model=api_model,
    temperature=0.1,
    messages=[
      {"role": "system", "content": "Using the given documentation delimited by triple quotes and current api state encoded in JSON, respond to user requests with the path(s) and value(s) you would modify in the api to achieve the users desired action in the format \n```{\"path\": path, \"value\": value}```. If the user request isn't possible with the given data respond with 'Not possible'"},
      {"role": "user", "content": '"""' + docs_by_name[current_key] + '""" \n\n' + data + '\n\nrequest: ' + query},
    ]
  )

  print(completion.choices[0].message.content)
  mutate_command = completion.choices[0].message.content
  if not 'Not possible' in mutate_command:
    print(mutate_command)
    break

{"path": "data.map1.defaultViewport.latitude", "value": 35.86166}
{"path": "data.map1.defaultViewport.longitude", "value": 104.195397}
{"path": "data.map1.defaultViewport.latitude", "value": 35.86166}
{"path": "data.map1.defaultViewport.longitude", "value": 104.195397}
